# Lógica de primer orden
Para trabajar con lógica de primer orden, vamos a utilizar el paquete [nltk](http://nltk.org), y en particular su módulo de inferencia lógia y de construcción de modelos.

In [1]:
from nltk import *

In [2]:
from nltk.inference.resolution import *

## Forma normal de Skolem

Para leer una expresión utilizamos `Expression.fromstring`, como lo vamos a utilizar mucho, le vamos a poner un nombre

- `all` y `exists` segido de variable y un punto significan para todo y existe, respectivamente
- las conectivas son `&` para el y, `|` para el o, `->` para el implica, `<->` para la equivalencia
- la negación es `-` 

In [3]:
expr = Expression.fromstring

El comando `skolemize` encuentra una forma normal de Skolem

In [4]:
e=expr("all x.S(x)->exist z.all y. R(x,y)")
skolemize(e)

<OrExpression (-S(z1) | R(x,z2))>

Para reducir los contadores de las variables en esa expresión usamos `normalize`

In [5]:
_.normalize()

<OrExpression (-S(z3) | R(z3,z3))>

In [6]:
e=expr("all x.(R(x,y)&-all y.R(x,y))")

In [7]:
s=skolemize(e).normalize()

In [8]:
print(s)

(R(z2,z1) & -R(z2,F4(z2)))


## Unificación

Dadas dos expresiones, `most_general_unification` proporciona un unificador más general

La salida es un diccionario con la substitución 

In [9]:
e1=expr("F(x,y)")
e2=expr("F(a,F(b))")
most_general_unification(e1,e2)

{x: a, y: F(b)}

## Cláusulas, forma clausular

El comando `Clause` aplicado a una lista de literales da como resultado una cláusula

In [10]:
Clause([e1,e2])

{F(x,y), F(a,F(b))}

También podemos utilizar `clausify` con una expresión obtenida previamente

Por ejemplo, si se lo aplicamos a $e=\forall x(R(x,y)\wedge \neg \forall y R(x,y))$, primero skolemiza y luego pasa a forma clausular

In [11]:
clausify(e)

[{R(z9,z8)}, {-R(z10,F6(z10))}]

In [12]:
e=expr("all x.P(x)-> exists x.Q(x)")
clausify(e)

[{-P(z13), Q(z14)}]

Existe un error que impide usar predicados sin variables con ciertos comandos, tal y como se ha informado [aquí](https://github.com/nltk/nltk/issues/37)

In [13]:
e=expr("Q|-Q")

El comando `skolemize(e)`, y por tanto `clausify(e)` daría un error (que hemos omitido aquí)

## Unificación y resolución

Las cláusulas tienen un método que permite calcular resolventes

In [14]:
c1=Clause([expr("P(x)"),expr("Q(a)")])
c2=Clause([expr("-P(a)")])

In [15]:
c1

{P(x), Q(a)}

In [16]:
c2

{-P(a)}

In [17]:
c1.unify(c2)

[{Q(a)}]

También podemos determinar si una cláusula es una tautología

In [18]:
c=clausify(expr("all x.(P(x)->P(x))"))
c

[{-P(z16), P(z16)}]

In [19]:
c[0].is_tautology()

True

O si una cláusula es subcláusula (subsume) a otra

In [20]:
Clause([expr("P(x)")]).subsumes(Clause([expr("P(x)"), expr("Q(A)")]))

True

## Demostraciones por resolución

Podemos utilizar `ResolutionProverComman` para realizar demostraciones por resolución

In [21]:
p1 = expr('humano(socrates)')
p2 = expr('all x.(humano(x) -> mortal(x))')
c  = expr('mortal(socrates)')
print(p1,", ",p2,", ",c)

humano(socrates) ,  all x.(humano(x) -> mortal(x)) ,  mortal(socrates)


In [22]:
demostracion =  ResolutionProverCommand(c, [p1,p2])

In [23]:
demostracion.prove()

True

In [24]:
print(demostracion.proof())

[1] {-mortal(socrates)}          A 
[2] {humano(socrates)}           A 
[3] {-humano(z18), mortal(z18)}  A 
[4] {-humano(socrates)}          (1, 3) 
[5] {mortal(socrates)}           (2, 3) 
[6] {}                           (1, 5) 



Esta demostración nos indica que los tres primeros pasos son elementos de $\{p_1, p_2,\neg c\}$, que [4] es resolvente de 1 y 3, que [5] lo es de 2 y 3, y que finalmente se obtiene la cláusula vacía como resolvente de 1 y 5

Veamos otro ejemplo, ahora con funciones

In [25]:
h1=expr("all y.-R(A,y)")
h2=expr("all x.(-Q(x)|R(x,F(x)))")
c=expr("-Q(A)")

In [26]:
demostracion =  ResolutionProverCommand(c, [h1,h2])

In [27]:
demostracion.prove()

True

In [28]:
print(demostracion.proof())

[1] {Q(A)}                    A 
[2] {-R(A,z20)}               A 
[3] {-Q(z22), R(z22,F(z22))}  A 
[4] {R(A,F(A))}               (1, 3) 
[5] {-Q(A)}                   (2, 3) 
[6] {}                        (1, 5) 



**Ejemplo con error**

Veamos ahora un ejemplo más complicado:
$$\{ \forall y(-C(y)\to \exists x A(x,y)), \forall x(\exists y(-C(y)\wedge A(x,y))\to M(x)), \forall x(D(x)\to M(x)), \forall x((M(x)\wedge  D(x))\to \neg(\exists y(\neg C(y)\wedge A(x,y)))), \exists x\neg C(x)\}\models \exists x(M(x)\wedge \neg D(x)) $$

In [29]:
a=expr("exists x.(M(x)&-D(x))")

In [30]:
print(a)

exists x.(M(x) & -D(x))


In [31]:
h1=expr("all y.(-C(y)->exists x.A(x,y))")
h2=expr("all x.(exists y.(-C(y)&A(x,y))->M(x))")
h3=expr("all x.(D(x)->M(x))")
h4=expr("all x.((M(x)&D(x))->-(exists y.(-C(y)&A(x,y))))")
h5=expr("exists x.-C(x)")

In [32]:
demostracion =  ResolutionProverCommand(a, [h1,h2,h3,h4,h5])

In [33]:
demostracion.prove()

True

In [34]:
print(demostracion.proof())

[1] {-M(z24), D(z24)}                        A 
[2] {C(z27), A(F25(z27),z27)}                A 
[3] {C(z30), -A(z31,z30), M(z31)}            A 
[4] {-D(z33), M(z33)}                        A 
[5] {-M(z37), -D(z37), C(z36), -A(z37,z36)}  A 
[6] {-C(z39)}                                A 
[7] {C(z30), -A(z31,z30), D(z31)}            (1, 3) 
[8] {}                                       (1, 4) 



Si observamos la demostración que nos proporciona este comando, vemos que de [1] y [4] ha obtenido la cláusula vacía, cosa que no es posible. Algo va mal. Veamos si podemos acotar mejor el error. Para ello tomemos el siguiente conjunto de fórmulas

In [35]:
h = Expression.fromstring("all x.(P(x) | Q(x))")
t= Expression.fromstring("all x.(-P(x) | -Q(x))")
s = Expression.fromstring("exists x.R(x)")
demostracion=ResolutionProverCommand(s,[h,t])

In [36]:
demostracion.prove()

True

In [37]:
print(demostracion.proof())

[1] {-R(z41)}           A 
[2] {P(z43), Q(z43)}    A 
[3] {-P(z45), -Q(z45)}  A 
[4] {}                  (2, 3) 



Una vez más vemos que deduce la cláusula vacía como resolvente de [2] y [3], lo cual no es posible

Veamos cómo hace la resolvente en estos casos

In [38]:
c=Clause([expr("P(x)"),expr("Q(x)")])
cn=Clause([expr("-P(x)"),expr("-Q(x)")])

In [39]:
c.unify(cn)

[{}]

Lo cual es claramente erróneo

## Demostración usando Tablau

In [40]:
demostracion = TableauProverCommand(s,[h,t])

In [41]:
demostracion.prove()

False

In [42]:
print(demostracion.proof())

-exists x.R(x)
   all x.(-P(x) | -Q(x)):   []
         --> Using 'z46'
      (-P(z46) | -Q(z46))
         -P(z46)
            all x.(-P(x) | -Q(x)):   [z46]
                  --> Variables Exhausted
               all x.-R(x):   []
                     --> Using 'z46'
                  -R(z46)
                     all x.(-P(x) | -Q(x)):   [z46]
                           --> Variables Exhausted
                        all x.-R(x):   [z46]
                              --> Variables Exhausted
                           all x.(P(x) | Q(x)):   []
                                 --> Using 'z46'
                              (P(z46) | Q(z46))
                                 P(z46)
                                    CLOSED
                                 Q(z46)
                                    all x.(-P(x) | -Q(x)):   [z46]
                                          --> Variables Exhausted
                                       all x.-R(x):   [z46]
                                     

In [43]:
h1=expr("all y.(-C(y)->exists x.A(x,y))")
h2=expr("all x.(exists y.(-C(y)&A(x,y))->M(x))")
h3=expr("all x.(D(x)->M(x))")
h4=expr("all x.((M(x)&D(x))->-(exists y.(-C(y)&A(x,y))))")
h5=expr("exists x.-C(x)")
a=expr("exists x.(M(x)&-D(x))")

In [44]:
demostracion = TableauProverCommand(a,[h1,h2,h3,h4,h5])

Por desgracia `demostracion.prove()` demora más de lo deseado

In [45]:
h1=expr("all y.-R(A,y)")
h2=expr("all x.(-Q(x)|R(x,F(x)))")
c=expr("-Q(A)")
demostracion = TableauProverCommand(c,[h1,h2])

In [46]:
demostracion.prove()

RecursionError: maximum recursion depth exceeded while calling a Python object

## Demostraciones con prover9

Por suerte, `nltk` ofrece una interfaz a [prover9](http://www.cs.unm.edu/~mccune/mace4) que parece ser rápida y correcta

In [49]:
h1=expr("all y.(-C(y)->exists x.A(x,y))")
h2=expr("all x.(exists y.(-C(y)&A(x,y))->M(x))")
h3=expr("all x.(D(x)->M(x))")
h4=expr("all x.((M(x)&D(x))->-(exists y.(-C(y)&A(x,y))))")
h5=expr("exists x.-C(x)")
a=expr("exists x.(M(x)&-D(x))")

In [50]:
demostracion = Prover9Command(a,[h1,h2,h3,h4,h5])

In [51]:
demostracion.prove()

True

In [52]:
print(demostracion.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 30475 was started by pedro on MacBook-Pro-de-Pedro.local,
Thu May 12 17:52:50 2016
The command was "/Users/pedro/lib/LADR-2009-11A/bin/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.00 (+ 0.00) seconds.
% Length of proof is 17.
% Level of proof is 5.
% Maximum clause weight is 3.000.
% Given clauses 0.


1 (all x (-C(x) -> (exists y A(y,x)))).  [assumption].
2 (all x ((exists y (-C(y) & A(x,y))) -> M(x))).  [assumption].
4 (all x (M(x) & D(x) -> -(exists y (-C(y) & A(x,y))))).  [assumption].
5 (exists x -C(x)).  [assumption].
6 (exists x (M(x) & -D(x))).  [goal].
7 -C(c1).  [clausify(5)].
8 C(x) | A(f1(x),x).  [clausify(1)].